In [41]:
import sys
sys.path.append("../src")

In [43]:
from defense import Server_with_max_norm

ImportError: attempted relative import beyond top-level package

In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score

In [3]:
config = {
    "batch_size":128
}

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [5]:
raw_df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')

In [6]:
raw_df_neg = raw_df[raw_df["Class"] == 0]
raw_df_pos = raw_df[raw_df["Class"] == 1]

down_df_neg = raw_df_neg#.sample(40000)
down_df = pd.concat([down_df_neg, raw_df_pos])

In [7]:
neg, pos = np.bincount(down_df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 284807
    Positive: 492 (0.17% of total)



In [8]:
cleaned_df = down_df.copy()

# You don't want the `Time` column.
cleaned_df.pop('Time')

# The `Amount` column covers a huge range. Convert to log-space.
eps = 0.001 # 0 => 0.1¢
cleaned_df['Log Ammount'] = np.log(cleaned_df.pop('Amount')+eps)

In [9]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [10]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (182276,)
Validation labels shape: (45569,)
Test labels shape: (56962,)
Training features shape: (182276, 29)
Validation features shape: (45569, 29)
Test features shape: (56962, 29)


In [28]:
train_dataset = DataSet(train_features,
                        train_labels.astype(np.float64).reshape(-1, 1))
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config["batch_size"],
                                           shuffle=True)

test_dataset = DataSet(test_features,
                       test_labels.astype(np.float64).reshape(-1, 1))
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=config["batch_size"],
                                          shuffle=True)

In [29]:
hidden_dim = 16

class FirstNet(nn.Module):
    def __init__(self):
        super(FirstNet, self).__init__()        
        self.L1 = nn.Linear(train_features.shape[-1],
                            hidden_dim)

    def forward(self, x):
        x = self.L1(x)
        x = nn.functional.relu(x)
        return x
    
class SecondNet(nn.Module):
    def __init__(self):
        super(SecondNet, self).__init__()        
        self.L2 = nn.Linear(hidden_dim,
                            1)

    def forward(self, x):
        x = self.L2(x)
        x = torch.sigmoid(x)
        return x
    
model_1 = FirstNet()
model_1 = model_1.to(device)

model_2 = SecondNet()
model_2 = model_2.to(device)

model_1.double()
model_2.double()

SecondNet(
  (L2): Linear(in_features=16, out_features=1, bias=True)
)

In [ ]:
def Max_Norm(client):
    class client(Client):

In [30]:
criterion = nn.BCELoss()

opt_1 = optim.Adam(model_1.parameters(), lr=1e-3)
opt_2 = optim.Adam(model_2.parameters(), lr=1e-3)

In [31]:
client = Client(model_1, opt_1)
server = Server(model_2, opt_2, criterion)

In [32]:
def torch_auc(label, pred):
    return roc_auc_score(label.detach().numpy(),
                         pred.detach().numpy())

In [33]:
sn = SplitNN(client, server, device=device)

In [34]:
sn.fit(train_loader, 10, metric=torch_auc)

epoch 1, loss 0.00057904, metric 0.8411499868114477
epoch 2, loss 3.6767e-05, metric 0.9357675554285548
epoch 3, loss 2.9641e-05, metric 0.9621937141894169
epoch 4, loss 2.7405e-05, metric 0.9704612238243869
epoch 5, loss 2.6088e-05, metric 0.9760471802142408
epoch 6, loss 2.473e-05, metric 0.9790083857211939
epoch 7, loss 2.319e-05, metric 0.9822661046877977
epoch 8, loss 2.1743e-05, metric 0.9862272680646533
epoch 9, loss 1.9893e-05, metric 0.9882808080919095
epoch 10, loss 1.8333e-05, metric 0.9892892835685291


In [ ]:
class LLSplitNN(SplitNN):
    def __init__(self, server_model,
                server_optimizer,
                criterion):
        super().__init__(server_model,
                server_optimizer,
                criterion)    
        
    def _attack_